# **Deteksi Outlier dengan metode Local Outlier Factor (LOF) dalam Data Understanding**
## APA ITU LOF
Local Outlier Factor (LOF) adalah metode berbasis kepadatan yang digunakan untuk mendeteksi outlier dengan membandingkan kepadatan lokal suatu titik data dengan kepadatan lokal tetangganya. Jika kepadatan suatu titik jauh lebih rendah dibandingkan dengan tetangganya, maka titik tersebut dianggap sebagai outlier.

Keunggulan LOF dibanding metode berbasis jarak lainnya seperti KNN Distance adalah kemampuannya menangani dataset dengan kepadatan yang bervariasi, sehingga lebih akurat dalam mendeteksi outlier pada data yang tidak terdistribusi secara merata.

## BAGAIMANA TAHAPAN LOF
Tahapan Local Outlier Factor (LOF):

**1. Persiapan Data**
- Impor dataset yang akan dianalisis.
- Pastikan data bersih dan tidak mengandung nilai yang hilang.
- Jika perlu, lakukan normalisasi atau standarisasi agar skala variabel seragam.

**2. Tentukan Parameter K**
- Pilih jumlah tetangga terdekat (K).
- Biasanya, K dipilih berdasarkan aturan seperti sqrt(N) atau eksperimen dengan beberapa nilai K.
**3. Hitung Jarak Antar Titik**
- Gunakan metrik jarak seperti Euclidean Distance atau Manhattan Distance untuk mengukur seberapa jauh setiap titik dari tetangga terdekatnya.
**4. Tentukan Skor Outlier**
- Gunakan metrik seperti K-Nearest Neighbor Distance (KNN Distance), yaitu rata-rata jarak ke K tetangga terdekat.
- Atau gunakan Local Outlier Factor (LOF), yang membandingkan kerapatan lokal suatu titik dengan tetangganya.
**5. Tentukan Threshold**
- Gunakan percentile atau IQR untuk menentukan batas outlier.
- Atau tentukan threshold berdasarkan distribusi skor outlier.
**6. Identifikasi Outlier**
- Data dengan skor outlier lebih tinggi dari threshold dianggap outlier.
**7. Visualisasi Hasil**
- Gunakan scatter plot, box plot, atau histogram untuk melihat sebaran data dan outlier.
**8. Tindakan Lanjutan**
- Tinjau apakah outlier perlu dihapus, diubah, atau diteliti lebih lanjut.
- Jika digunakan untuk model, pertimbangkan apakah outlier memberi informasi penting atau hanya noise.
   
## CONTOH MENGHITUNG MANUAL LOF
Berikut adalah cara singkat menghitung **Local Outlier Factor (LOF)** untuk satu titik data dengan dua fitur:

1. **Dataset**:  
   Misalkan data berikut:

| ID | Feature1 | Feature2 |
|----|----------|----------|
| 1  | 2.0      | 3.0      |
| 2  | 3.0      | 4.0      |
| 3  | 4.0      | 5.0      |
| 4  | 5.0      | 6.0      |
| 5  | 6.0      | 7.0      |

   Kita akan hitung LOF untuk **titik 3** (4.0, 5.0).

2. **Hitung Jarak Euclidean**:  
   Jarak antara titik 3 dan titik lainnya:
   - Titik 3 ke Titik 2: 1.41
   - Titik 3 ke Titik 4: 1.41

3. **Tentukan k-Tetangga Terdekat** (k=2):  
   Titik 3 memiliki dua tetangga terdekat: titik 2 dan titik 4.

4. **Hitung Reachability Distance**:  
   Reachability distance antara titik 3 dan tetangga:
   - Titik 3 ke Titik 2: 1.41
   - Titik 3 ke Titik 4: 1.41

5. **Hitung Local Reachability Density (LRD)**:  
   $
   \text{LRD}(3) = \frac{1}{\frac{1}{2} \times (1.41 + 1.41)} = 0.71
   $

6. **Hitung LOF**:  
   $
   \text{LOF}(3) = \frac{0.71}{0.71} + \frac{0.71}{0.71} = 2
   $

7. **Interpretasi**:  
   Karena **LOF = 2** (lebih besar dari 1), titik 3 dianggap **outlier**.

LOF mengukur seberapa terisolasi titik dibandingkan tetangganya. Titik dengan LOF lebih besar dari 1 dianggap outlier.

## IMPLEMENTASI PAKAI SKLEARN UNTUK DATA CONTOH

In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor

# Dataset contoh sesuai dengan tabel awal
data = {
    'Feature1': [2.0, 3.0, 4.0, 5.0, 6.0],
    'Feature2': [3.0, 4.0, 5.0, 6.0, 7.0]
}

# Membuat DataFrame
df = pd.DataFrame(data)

# Inisialisasi model LOF dengan k=2 (2 tetangga terdekat)
lof = LocalOutlierFactor(n_neighbors=2)

# Fit model LOF dan prediksi label (1 untuk normal, -1 untuk outlier)
lof_labels = lof.fit_predict(df)

# Menambahkan hasil prediksi ke DataFrame
df['LOF Label'] = lof_labels

# Menampilkan hasil
print(df)

# Menampilkan jumlah outlier
num_outliers = (lof_labels == -1).sum()
print(f"\nJumlah outlier: {num_outliers}")

# Menampilkan data outlier
outliers = df[df['LOF Label'] == -1]
print("\nData Outlier:")
print(outliers)


   Feature1  Feature2  LOF Label
0       2.0       3.0          1
1       3.0       4.0          1
2       4.0       5.0          1
3       5.0       6.0          1
4       6.0       7.0          1

Jumlah outlier: 0

Data Outlier:
Empty DataFrame
Columns: [Feature1, Feature2, LOF Label]
Index: []


In [3]:
%pip install pymysql
%pip install psycopg2

Perintah %pip install pymysql dan %pip install psycopg2 digunakan untuk menginstal pustaka yang memungkinkan Python berinteraksi dengan database MySQL dan PostgreSQL. pymysql digunakan untuk menghubungkan dan beroperasi dengan database MySQL, sementara psycopg2 berfungsi untuk berkomunikasi dengan database PostgreSQL. Dengan menginstal kedua pustaka ini, aplikasi Python dapat melakukan berbagai operasi database seperti query, insert, update, dan delete. Selain itu, penggunaan %pip install dalam Jupyter Notebook memungkinkan instalasi pustaka langsung dari dalam lingkungan notebook tanpa perlu menggunakan terminal atau command prompt.

In [4]:
import psycopg2
import pymysql
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean

def get_pg_data():
    conn = psycopg2.connect(
        host="pg-34d4055e-posgressqlpendataaa.g.aivencloud.com",
        user="avnadmin",
        password="AVNS_IRjo23jQlvziN0aBC2-",
        database="defaultdb",
        port=11481
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM auli.postgree")
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    cursor.close()
    conn.close()
    return pd.DataFrame(data, columns=columns)

def get_mysql_data():
    conn = pymysql.connect(
        host="mysql-9b686fb-pendataa.g.aivencloud.com",
        user="avnadmin",
        password="AVNS_ZuFdVS1OQkmHx4P1Wtp",
        database="defaultdb",
        port=22825
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM irismysql")
    data = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]
    cursor.close()
    conn.close()
    return pd.DataFrame(data, columns=columns)

# Ambil data dari kedua database
df_postgresql = get_pg_data()
df_mysql = get_mysql_data()

# Gabungkan berdasarkan kolom 'id' dan 'Class'
df_merged = pd.merge(df_mysql, df_postgresql, on=["id", "class"], how="inner")

# Ambil data fitur numerik
feature_columns = ["petal_length", "petal_width", "sepal_length", "sepal_width"]
data_values = df_merged[feature_columns].values

Kode ini mengambil data dari dua sumber database yang berbeda, yaitu PostgreSQL dan MySQL, kemudian menggabungkannya untuk analisis lebih lanjut. Data diambil dari tabel auli.postgree di PostgreSQL dan irismysql di MySQL menggunakan pustaka psycopg2 dan pymysql. Setelah itu, kedua dataset digabungkan berdasarkan kolom id dan class menggunakan metode inner join, sehingga hanya data yang cocok dari kedua tabel yang dipertahankan. Selanjutnya, beberapa fitur numerik yang relevan, seperti petal_length, petal_width, sepal_length, dan sepal_width, diekstrak untuk keperluan analisis lebih lanjut. Dengan cara ini, kode menyiapkan dataset yang bersih dan terstruktur yang dapat digunakan untuk berbagai keperluan, seperti deteksi outlier atau analisis klasifikasi berdasarkan karakteristik bunga.









In [5]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor

# Gabungkan berdasarkan kolom 'id' dan 'class'
df_merge = pd.merge(df_mysql, df_postgresql, on=["id", "class"], how="inner")

# Ambil data fitur numerik tanpa kolom 'class'
feature_columns = ["petal_length", "petal_width", "sepal_length", "sepal_width"]
data_values = df_merge[feature_columns].values

# Inisialisasi model LOF
clf = LocalOutlierFactor(n_neighbors=90)
label = clf.fit_predict(data_values)

# Tambahkan hasil label ke dataframe
df_merge["outlier_label"] = label

# Cetak hasil dengan ID dan class
print(df_merge.to_string(index=False))

num_outliers = (label == -1).sum()
print(f"\nJumlah outlier: {num_outliers}")

outliers = df_merge[df_merge["outlier_label"] == -1]
print("\nData Outlier:")
print(outliers.to_string(index=False))

 id           class  petal_length  petal_width  sepal_length  sepal_width  outlier_label
  1     Iris-setosa           1.4          0.2           5.1          3.5              1
  2     Iris-setosa          14.0          2.0          40.9         30.0             -1
  3     Iris-setosa           1.3          0.2           4.7          3.2              1
  4     Iris-setosa           1.5          0.2           4.6          3.1              1
  5     Iris-setosa           1.4          0.2           5.0          3.6              1
  6     Iris-setosa           1.7          0.4           5.4          3.9              1
  7     Iris-setosa           1.4          0.3           4.6          3.4              1
  8     Iris-setosa           1.5          0.2           5.0          3.4              1
  9     Iris-setosa           1.4          0.2           4.4          2.9              1
 10     Iris-setosa           1.5          0.1           4.9          3.1              1
 11     Iris-setosa  

Kode ini menerapkan metode Local Outlier Factor (LOF) untuk mendeteksi outlier dalam dataset gabungan dari MySQL dan PostgreSQL. Dataset digabungkan berdasarkan kolom id dan class menggunakan metode inner join, sehingga hanya data yang cocok dari kedua tabel yang dipertahankan. Analisis dilakukan menggunakan fitur numerik petal_length, petal_width, sepal_length, dan sepal_width, yang kemudian diubah ke dalam bentuk array untuk deteksi outlier. Model LOF dengan parameter n_neighbors=90 diterapkan, di mana setiap titik dinilai berdasarkan kedekatannya dengan 90 tetangga terdekat. Model memberikan label -1 untuk titik yang dianggap sebagai outlier dan 1 untuk titik normal, lalu hasilnya ditambahkan ke dalam dataset. Jumlah total outlier dihitung dan ditampilkan, sementara data yang terdeteksi sebagai outlier dicetak untuk dianalisis lebih lanjut. Metode LOF ini efektif dalam mengidentifikasi anomali berdasarkan kepadatan lokal data, sehingga memungkinkan analisis lebih lanjut untuk memahami pola outlier dalam dataset.